# It is important to ask the right questions for the analysis to be done right. Under mentioned are the questions to be answered via various data analytics methods.

> > > 
> Research Questions
> 
> 
> 1. How do various strategies undertaken by various countries contribute to their respective winnings ?
> 2. To what extent does the host country have an advantage in the medals won at the Olympics? 
> 3. How baised is the Olympics? Is there a significant difference in the sex ratio of participants and winners and How has it changed over the years?
> 4. Is there an age group for winning the Olympics, or not? Or has it changed over the years?
> 5. Is there a consistency in events/ countries won?
> 6. Is any country dominating the Olympics?
> 7. Is there a pattern or a certain set of characteristics that might be help cluster the participants in the olympcis?
> 8. How did COVID affect the summer Olympics held in 2021?
> 9. How has the Olympics changed (growth) over the years?
> 10. Has India’s participation and performance in the Olympics over the years improved? 
>

In [5]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
#install.packages("psych")
library(psych);
library("ggpubr");
library(MASS);
library(dplyr)
library(fastDummies);
library(corrplot) #correlation plot
library(NbClust)
library(stats)
library(plotly)
library(ggrepel)
library(RColorBrewer)
library(dplyr)
library(viridis)
library(patchwork)
library(rpart)
library(e1071)
library(magrittr) # needs to be run every time you start R and want to use %>%
library(dplyr) 
library(naivebayes)
library(e1071)
library(nnet)
library(caret)
library(superml)
library(dbscan)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ERROR: Error in library("ggpubr"): there is no package called 'ggpubr'


Reading the Olympics (Main dataset) into 'olympics' variable.

In [ ]:
#getting the dataset
olympics <- read.csv("../input/120-years-of-olympic-history/athlete_events.csv")

In [ ]:
olympics_2021 <-read.csv("../input/olympic-games-2021-medals/Tokyo 2021 dataset v3.csv")

In [ ]:
olympics_data_dictionary <- read.csv("../input/120-years-of-olympic-history/athlete_events_data_dictionary.csv")

In [ ]:
olympics_winners <- olympics[c(which(olympics$Medal=='Gold'|olympics$Medal=='Silver'|olympics$Medal=='Bronze')),]

# First Task : 
This is also called as data wrangaling/ data preprocessing / data preparation.

Analyse the features of the dataset. That includes the following,
1. Describe the dataset
2. Identify the types of variables 
3. Change the types of variables if necessary 
4. Remove or modify Null variables
5. Identify outliers (if present)
6. Deal with outliers

In [ ]:
#dimentionality of the dataset
dim(olympics)

In [ ]:
nrow(olympics)

In [ ]:
ncol(olympics)

To get a breif idea about the dataset, let us first view the first n instances of the dataset.

In [ ]:
head(olympics,n=15)

In [ ]:
#Identifying the types of variables
glimpse(olympics)

#this provides a glimpse of the dataset. Various attributes, their respective types and a few instance examples.

Here, we can note that the Columns have their respective datatype. The only datatype that we can change so that the data analysis so that it would become convinient to the data analysis is - Weight, which is currently in 'dbl' datatype which represents double. It can be converted into 'int', that is Integer type.

In [ ]:
#changing the Datatype of the attributes according to convinience. 
#transform(olympics, Weight = as.integer(Weight))
head(transform(olympics, Weight = as.integer(Weight)))

#change categorical data into factors for ease in analysis.
################olympics$Sex <- factor(olympics$Sex)

#similarly other categirucal values can be changed as well. 
# medals are ordinal data, and therefore levels can be providd as well.
################olympics$Medal <- factor(olympics$Medal, levels = c(NA,'Bronze','Silver','Gold'), ordered = TRUE)

#To view the affects on the type of attribute.
################str(olympics)

Now lets' use a predefined function - summary, to find out the summary of the dataset

In [ ]:
summary(olympics)

As you can see above, there are vaious null values in the following attributes : Age, Height, Weight and Medal. 
The NA in Medal attribute only refers to no medal recieved, and can be modified easily. Other attributes' null values must be handled with.

Now let's individually review the null variables in each attribute and work on them.

In [ ]:
sum(length(which(is.na(olympics$Age))))

In [ ]:
olympics$Age[c(which(is.na(olympics$Age)))]<- mean(olympics$Age, na.rm=TRUE)

In [ ]:
mean(olympics$Age)

In [ ]:
sum(length(which(is.na(olympics$Age))))

Now we've dealt with Age factor. Now lets' move on to the missing values in Height and Weight.

In [ ]:
mean(is.na(olympics$Weight))

In [ ]:
mean(is.na(olympics$Height))

In [ ]:
which(is.na(olympics$Height) & is.na(olympics$Weight) & is.na(olympics$Medal))

These are the indices of the rows that do not have both the weight and the height of the participant and havent won any medal either

In [ ]:
#now we can delete these instances, because compared to the orignial dataset, these are not significant.

In [ ]:
olympics <- olympics[-c(which(is.na(olympics$Height) & is.na(olympics$Weight) & is.na(olympics$Medal))),]

Now lets' check if they have been dropped.

In [ ]:
sum(which(is.na(olympics$Height) & is.na(olympics$Weight) & is.na(olympics$Medal)))

In [ ]:
sum(is.na(olympics$Medal))

We'll have to change NA to 'No Medal'

In [ ]:
olympics$Medal[c(which(is.na(olympics$Medal)))]<- 'No Medal'

In [ ]:
sum(is.na(olympics$Medal))

Now lets' run the summary again and have a look at the statistics.

In [ ]:
summary(olympics)

In [ ]:
sum(is.na(olympics$ID))
sum(is.na(olympics$Name))
sum(is.na(olympics$Sex))
sum(is.na(olympics$Age))
sum(is.na(olympics$Height))
sum(is.na(olympics$Weight))
sum(is.na(olympics$Team))
sum(is.na(olympics$NOC))
sum(is.na(olympics$Games))
sum(is.na(olympics$Year))
sum(is.na(olympics$Season))
sum(is.na(olympics$City))
sum(is.na(olympics$Sport))
sum(is.na(olympics$Event))
sum(is.na(olympics$Medal))

In [ ]:
olympics <- na.omit(olympics)

In [ ]:
summary(olympics)

Identify outliers.

In [ ]:
outlier_values <- boxplot.stats(olympics$Height)$out 
boxplot(olympics$Height, main=" Height", boxwex=0.1)
mtext(paste("Outliers: ", paste(outlier_values, collapse=", ")), cex=0.6)

In [ ]:
outlier_values <- boxplot.stats(olympics$Age)$out 
boxplot(olympics$Age, main=" Age", boxwex=0.1)
mtext(paste("Outliers: ", paste(outlier_values, collapse=", ")), cex=0.6)

Although in most statistics outliers prove to be non-benefical or non-crutial, for our analysis, outliers are possible.

In [ ]:
olympics[c(which(olympics$Age==11)),]

> Liana Vicens is a Puerto Rican former swimmer. She competed in four events at the 1968 Summer Olympics. At 11 years old, she remained the youngest known competitor in the history of the Olympic Games as of 2016 [Here](https://en.wikipedia.org/wiki/Liana_Vicens)

> Sonja Henie made her Olympic debut at the first Olympic Winter Games in Chamonix in 1924, at the time, she was only 11 years old. [here](https://olympics.com/en/athletes/sonja-henie)

> [At the age of 11 years and 158 days, she competed at the 1968 Winter Olympics in Grenoble, France.](http://en.wikipedia.org/wiki/Beatrice_Hu%C8%99tiu)

In [ ]:
olympics[c(which(olympics$Age==71)),]

> Hoketsu, 71, was the oldest competitor in both Beijing in 2008 and London 2012, where he finished 17th in a field of 24 in the men’s dressage competition,­ almost half a century after making his Games debut in Tokyo in 1964. [Here ](https://olympics.com/en/athletes/hiroshi-hoketsu)

# Exploratory Data Analysis
1. Understand and draw inferences from each attribute.

In [ ]:
dim(olympics_data_dictionary)
print(olympics_data_dictionary)

The data dictionary provided gives us a brief understanding of what each attribute refers to.

In [ ]:
summary(olympics)

To view the number of unique values in each attribute.

In [ ]:
#ID
length(unique(olympics$ID))

The following value represents the total number of people that have participated in the olympics for the past 120 years.

In [ ]:
#Name
length(unique(olympics$Name))

Over 99 thousand people have participated in the olympic games in the past 120 years.

In [ ]:
#Sex
unique(olympics$Sex)

In [ ]:
no_of_male_participations = length(c(which(olympics$Sex == 'M')))

In [ ]:
no_of_female_participations = length(c(which(olympics$Sex == 'F')))

In [ ]:
# 140124+66729 = 206853

Sex Distribution

In [ ]:
pie(c(no_of_male_participations,no_of_female_participations),c("male","female"), main = "Male VS Female Participation")

As we can see clearly, the female population is way less than half of the participation by male. It is almost thrice.

The following is the range of age of the participants of the Olympics for the past 120 years.

In [ ]:
olympics_female_data <- olympics[c(which(olympics$Sex == 'F')),]

In [ ]:
olympics_male_data <- olympics[c(which(olympics$Sex == 'M')),]

In [ ]:
dim(olympics_female_data)

In [ ]:
no_of_female_participants <- length(unique(olympics_female_data$Name))
no_of_female_participants

In [ ]:
no_of_male_participants <- length(unique(olympics_male_data$Name))
no_of_male_participants

In [ ]:
barplot(c(no_of_male_participants,no_of_female_participants), 
       main = "Male VS Female participants",
        xlab = "Male And Female",
        ylab = "Count",
        names.arg = c("Male","Female"),
       )

In [ ]:
print(no_of_male_participations/no_of_male_participants)
print(no_of_female_participations/no_of_female_participants)

This is the ratio of the number of participantions to the number of total participants grouped by sex. 
The ratio is higher in the case of women, although less in number of participants in total, they take part in almost 2.25 times events. Whereas men take part in 2.02 times their total number.

So the total number of female participants is over 29 thousand. And they've participated in over 66 thousand events.

In [ ]:
no_of_male_participants <- length(unique(olympics_male_data$Name))
no_of_male_participants

Similarly, repeating the same for men, they have participated in over 140 thousand events with participants over 69 thousand.

Now lets' view the male and female participants ratio in the form of a pie chart to get a better understanding at a glance.

In [ ]:
print("male percentage :")
print((no_of_male_participations/(no_of_female_participations+no_of_male_participations))*100)
print("female percentage :")
print((no_of_female_participations/(no_of_female_participations+no_of_male_participations))*100)

In [ ]:
pie(c(no_of_male_participants,no_of_female_participants),c("male","female"), main = "Male VS Female Participants")

In [ ]:
print("male percentage :")
print((no_of_male_participants/(no_of_male_participants+no_of_female_participants))*100)
print("female percentage :")
print((no_of_female_participants/(no_of_male_participants+no_of_female_participants))*100)

Comparing the participants and the participation ratios, although there is a greater ratio of men participants, there is a lower ratio when it comes to participation. This means that female population although less in population is participating in multiple events.

Age Distribution

In [ ]:
#Age
max(olympics$Age)
min(olympics$Age)
range(olympics$Age)

Skewness

Skewness is the degree to which returns are asymmetric around the mean. Since a normal distribution is symmetric around the mean, skewness can be taken as one measure of how returns are not distributed normally. 
Visualizations of skewness more illuminating than the numbers themselves

In [ ]:
x <- olympics$Age
# Print skewness of distribution
#print(skewness(x))
# Histogram of distribution
hist(x)

> Looking at the visual histogram, it is evident that the age of the participants is skewed towards the left. That is, the participants are fairly young in most cases.

The same can be performed for the winners as well.

In [ ]:
x <- olympics_winners$Age
# Print skewness of distribution
#print(skewness(x))
# Histogram of distribution
hist(x)

The visual is very similar to that of the participations, it is skewed towards left.

Height Distribution

In [ ]:
x <- olympics$Height
# Print skewness of distribution
#print(skewness(x))
# Histogram of distribution
hist(x)

In order to find the correlation between various attributes they need to be numerical values.

Weight Distribution

In [ ]:
x <- olympics$Weight
# Print skewness of distribution
#print(skewness(x))
# Histogram of distribution
hist(x)

In [ ]:
M = cor(olympics[c(4,5,6)]) #correlation between age, height and weight. 
corrplot(M, method = 'shade', order = 'AOE', diag = FALSE)

In [ ]:
plot(x = olympics$Height,y = olympics$Weight,
   xlab = "Height",
   ylab = "Weight",		 
   main = "Height VS Weight",
    col = factor(olympics$Sex)
)

legend("topleft",
       legend = levels(factor(olympics$Sex)),
       pch = 10,
       col = factor(levels(factor(olympics$Sex))))

In [ ]:
# Calculate the mean age, separately for each Country.
age_mean_by_country <- aggregate(formula = Age ~ NOC,
          data = olympics,
          FUN = mean)
col(age_mean_by_country)
#hist(age_mean_by_country)

In [ ]:
print(length(unique(olympics$NOC)))

There are the various countries participating.

In [ ]:
length(unique(olympics$Team))

In [ ]:
olympics_selective <- olympics[c(which(olympics$NOC=='IND'|olympics$NOC=='CHN'|olympics$NOC=='NED')),]
#print(olympics_selective_countries)
dim(olympics_selective)
print(unique(olympics_selective$NOC))
boxplot(Height~NOC,data=olympics_selective, main=" Height", boxwex=0.1)

###### Conclussions and lessons
- The sex ratio is very biased.
- Although majority of the participants lie in the age group of 20-35, there are significant number of people in other age groups. The range of the participants is from 11 to 71.

In [ ]:
gold_winners <- olympics[c(which(olympics$Medal=='Gold')),]

In [ ]:
head(gold_winners)
#similarly for silver and bronze winners

silver_winners <- olympics[c(which(olympics$Medal=='Silver')),]
bronze_winners <- olympics[c(which(olympics$Medal=='Bronze')),]

In [ ]:
barplot(table(gold_winners$Age),
main="Age amoung gold winners",
xlab="Age",
ylab="Count",
border="red",
col="blue",
)

In [ ]:
gold_data = olympics %>%
    filter(Medal == 'Gold') %>% # Only keep entries about gold medals
    group_by(Team) %>% # Create a group for each country
    summarize(gold_count = n()) %>% # Count how many gold medals the country won.
                                    # Store it in a "gold_count" column.
    top_n(n=10, wt=gold_count) %>% # Only keep top 10 countries.
    arrange(desc(gold_count)) # Sort the data by number of gold medals.

# Print what we just computed
gold_data

In [ ]:
ggplot(data = gold_data, aes(x = Team, y = gold_count)) +
       geom_bar(stat = "identity") +
       coord_flip()

In [ ]:
#similarly silver and bronze 
silver_data = olympics %>%
    filter(Medal == 'Silver') %>% # Only keep entries about gold medals
    group_by(Team) %>% # Create a group for each country
    summarize(silver_count = n()) %>% # Count how many gold medals the country won.
                                    # Store it in a "gold_count" column.
    top_n(n=10, wt=silver_count) %>% # Only keep top 10 countries.
    arrange(desc(silver_count)) # Sort the data by number of gold medals.

# Print what we just computed
silver_data

In [ ]:
bronze_data = olympics %>%
    filter(Medal == 'Bronze') %>% # Only keep entries about gold medals
    group_by(Team) %>% # Create a group for each country
    summarize(bronze_count = n()) %>% # Count how many gold medals the country won.
                                    # Store it in a "gold_count" column.
    top_n(n=10, wt=bronze_count) %>% # Only keep top 10 countries.
    arrange(desc(bronze_count)) # Sort the data by number of gold medals.

# Print what we just computed
bronze_data

In [ ]:
ggplot(data = silver_data, aes(x = Team, y = silver_count)) +
       geom_bar(stat = "identity") +
       coord_flip()

In [ ]:
ggplot(data = bronze_data, aes(x = Team, y = bronze_count)) +
       geom_bar(stat = "identity") +
       coord_flip()

It is very evidnt that United states and soviet union has been ruling over all three medals in the olympics.

Now let's see the trend of India over the years.

In [ ]:
year_wise_india_count = olympics %>%
    #filter(Team) %>% # Only keep data for some countries
    filter(Medal %in% c('Gold', 'Silver', 'Bronze')) %>%
    filter(Team %in% c('India')) %>%
    group_by(Year, Team) %>% # Create groups on tuples!
    summarize(count=n()) %>% # Count how many gold medals the country won.
                                    # Store it in a "gold_count" column.
    top_n(n=1, wt=count) %>% # Only keep top 10 countries.
    arrange(desc(Year)) # Sort the data by number medals

# Print what we just computed
year_wise_india_count

Similarly, let's see the number of participants in India over the years.

In [ ]:

year_wise_india_count = olympics %>%
    #filter(Team) %>% # Only keep data for some countries
    filter(Medal %in% c('No Medal','Gold', 'Silver', 'Bronze')) %>%
    filter(Team %in% c('India')) %>%
    group_by(Year, Team) %>% # Create groups on tuples!
    summarize(count=n()) %>% # Count how many gold medals the country won.
                                    # Store it in a "gold_count" column.
    top_n(n=1, wt=count) %>% # Only keep top 10 countries.
    arrange(desc(Year)) # Sort the data by number medals

# Print what we just computed
year_wise_india_count

In [ ]:
bronze_data = olympics %>%
    filter(Medal == 'Bronze',) %>% # Only keep entries about gold medals
    group_by(Team) %>% # Create a group for each country
    summarize(bronze_count = n()) %>% # Count how many gold medals the country won.
                                    # Store it in a "gold_count" column.
    top_n(n=10, wt=bronze_count) %>% # Only keep top 10 countries.
    arrange(desc(bronze_count)) # Sort the data by number of gold medals.

# Print what we just computed
bronze_data

In [ ]:
p <- ggplot(olympics %>%
        filter(Team %in% c('India','Canada', 'United States', 'Germany', 'Sweden')) %>% # Only keep data for some countries
        filter(Medal %in% c('Gold', 'Silver', 'Bronze')) %>%
        group_by(Team, Medal, Year) %>% # Create groups on tuples!
        summarize(count=n(), .groups = 'drop')) + # Compute a count for each group
    geom_line(aes(x=Year, y=count, color=Team)) + # Add lines
    facet_wrap(~ Medal) + # Plot data of different Team on different facets
    scale_color_viridis(discrete=TRUE) + # colorblind-proof colors
    theme_bw()+ NULL
print(p)

In [ ]:
p <- ggplot(olympics %>%
        filter(Team %in% c('India','United States')) %>% # Only keep data for some countries
        filter(Medal %in% c('Gold', 'Silver', 'Bronze')) %>%
        group_by(Team, Medal, Year) %>% # Create groups on tuples!
        summarize(count=n(), .groups = 'drop')) + # Compute a count for each group
    geom_line(aes(x=Year, y=count, color=Team)) + # Add lines
    facet_wrap(~ Medal) + # Plot data of different Team on different facets
    scale_color_viridis(discrete=TRUE) + # colorblind-proof colors
    theme_bw()+ NULL
print(p)

The above mentioned is a comparitive plot of the team that has been ruling over the olympics with Indian Team. The winning streak started later as compared to that of the United states. Altough it started of well, over the years the winnings have taken a turn downwards. But recently initiatives have been taken to encourage Indians to participate more.

In [ ]:
p <- ggplot(olympics %>%
        filter(Team %in% c('India')) %>% # Only keep data for some countries
        filter(Medal %in% c('Gold')) %>%
        group_by(Team, Medal, Year) %>% # Create groups on tuples!
        summarize(count=n(), .groups = 'drop')) + # Compute a count for each group
    geom_line(aes(x=Year, y=count, color=Team)) + # Add lines
    facet_wrap(~ Medal) + # Plot data of different Team on different facets
    scale_color_viridis(discrete=TRUE) + # colorblind-proof colors
    theme_bw()+ NULL
print(p)

In [ ]:
p <- ggplot(olympics %>%
        filter(Team %in% c('India')) %>% # Only keep data for some countries
        filter(Medal %in% c('Silver')) %>%
        group_by(Team, Medal, Year) %>% # Create groups on tuples!
        summarize(count=n(), .groups = 'drop')) + # Compute a count for each group
    geom_line(aes(x=Year, y=count, color=Team)) + # Add lines
    facet_wrap(~ Medal) + # Plot data of different Team on different facets
    scale_color_viridis(discrete=TRUE) + # colorblind-proof colors
    theme_bw()+ NULL
print(p)

In [ ]:
p <- ggplot(olympics %>%
        filter(Team %in% c('India')) %>% # Only keep data for some countries
        filter(Medal %in% c('Bronze')) %>%
        group_by(Team, Medal, Year) %>% # Create groups on tuples!
        summarize(count=n(), .groups = 'drop')) + # Compute a count for each group
    geom_line(aes(x=Year, y=count, color=Team)) + # Add lines
    facet_wrap(~ Medal) + # Plot data of different Team on different facets
    scale_color_viridis(discrete=TRUE) + # colorblind-proof colors
    theme_bw()+ NULL
print(p)

In [ ]:
# Globally, how medals are distributed among the previous countries?
p <- ggplot(olympics %>%
        filter(Team %in% c('India','Australia','China','Russia', 'United States', 'Germany', 'Soviet Union', 'Japan')) %>% # Only keep entries from the selected countries.
        filter(Medal %in% c('Gold', 'Silver', 'Bronze'))) + # Only keep entries that got a medal.
    geom_bar(aes(x=Team, fill=Medal)) + # Add an histogram (bar chart about distribution) in the plot
    scale_fill_manual(values=c("#824A02", "#FEE101", "#D7D7D7")) + # Custom gold, silver and bonze colors colors
    theme_bw() + NULL
print(p)

In [ ]:
print(unique(olympics$Medal))

In [ ]:
# Globally, how medals are distributed among the previous countries?
p <- ggplot(olympics %>%
        filter(Team %in% c('India','Australia','China','Russia', 'United States', 'Germany', 'Soviet Union', 'Japan')) %>% # Only keep entries from the selected countries.
        filter(Medal %in% c('No Medal','Gold', 'Silver', 'Bronze'))) + # Only keep entries that got a medal.
    geom_bar(aes(x=Team, fill=Medal)) + # Add an histogram (bar chart about distribution) in the plot
    scale_fill_manual(values=c("#824A02", "#FEE101","#888888", "#D7D7D7")) + # Custom gold, silver and bonze colors colors
    theme_bw() + NULL
print(p)

Here, we've included the 'no medal' as well to have a clear idea of the number of participants from each country as well.

In [ ]:
year_wise_count = olympics %>%
    #filter(Team) %>% # Only keep data for some countries
    filter(Medal %in% c('Gold', 'Silver', 'Bronze')) %>%
    group_by(Year, Team) %>% # Create groups on tuples!
    summarize(count=n()) %>% # Count how many gold medals the country won.
                                    # Store it in a "gold_count" column.
    top_n(n=1, wt=count) %>% # Only keep top 10 countries.
    arrange(desc(Year)) # Sort the data by number of gold medals.

# Print what we just computed
year_wise_count

In [ ]:
games_plot <- olympics %>% count(Games)
games_plot

In [ ]:

coul <- brewer.pal(5, "Set2") 

barplot(games_plot$n,
       names.arg = games_plot$Games,
       col=coul)

In [ ]:
years_plot <- olympics %>% count(Year)
years_plot

In [ ]:
barplot(years_plot$n,
       names.arg = years_plot$Year,
       col=coul,
       las=3)

Number of games over the years increased.

In [ ]:
City_plot <- olympics %>% count(City)
City_plot

In [ ]:
Sport_plot <- olympics %>% count(Sport)
Sport_plot

In [ ]:
barplot(Sport_plot$n,
       names.arg = Sport_plot$Sport,
       col=coul,
       las=3)

In [ ]:
sport_data = olympics %>%
    group_by(Sport) %>% # Create a group for each Sport
    summarize(sport_count = n()) %>% 
                                    
    top_n(n=10, wt=sport_count) %>% 
    arrange(desc(sport_count)) #

# Print what we just computed
sport_data

In [ ]:
ggplot(data = sport_data, aes(x = Sport, y = sport_count)) +
       geom_bar(stat = "identity") +
       coord_flip() 
#top 10 sports in olympics over the years

In [ ]:
print(length(unique(olympics$Year)))

Over a course of 35 olympics, United States secured the highest number of Gold medals. That is 29 times. Which evidently proves the fact that United States dominates over the olympics. And only recently canada started catching up. Soviet Union has providid United Nations a tough competition over the years.

As you can see from the plot above, this is the winning over the years of a certain few teams.

Clustering Visualization Techniques.

In [ ]:
features <- olympics[,c(3:6,10,13)]
features <- features[complete.cases(features), ] 
plt0 <- features %>% filter(Year >= 1924) %>% ggplot(aes(x = Height, y = Weight)) + geom_density_2d(aes(color = as.factor(Sex))) + facet_wrap(.~Year)
plt0

This cluster plot show the height and weight plot over the years. This is a time series analysis.

In [ ]:
features %>% filter(Year == 2004) %>% filter(Sport == "Weightlifting") %>% ggplot(aes(x = Height, y = Weight)) + geom_density_2d(aes(color = as.factor(Sex))) + xlim(c(120, 215)) + ylim(30, 130)
features %>% filter(Year == 2004) %>% filter(Sport == "Volleyball") %>% ggplot(aes(x = Height, y = Weight)) + geom_density_2d(aes(color = as.factor(Sex))) + xlim(c(120, 215)) + ylim(30, 130)

This geom density graph depicts the density of a particular 'height' and 'weight' among various participants.

Naturally, the various volleyball players are taller, however the  weightlifters vary a lot in weight as they have different weight categoris, which makes sense. 
Following this,let's go to the clustering. Starting with k-means, 20 initial clusters. And, taking averages per category.

In [ ]:

options(ggrepel.max.overlaps = Inf)
features <- features %>% group_by(Sport) %>% sample_n(25, replace =  TRUE)

features <- features %>% mutate(Sex = ifelse(Sex == "M",1,0))
cfeat <- features[,c(1:4)]
clust <- kmeans(scale(cfeat), 20)
labels <- clust$cluster
cfeat <- cfeat %>% mutate(labels = labels) %>% mutate(Year = features$Year) %>% mutate(Sport = features$Sport) %>% mutate(Age = features$Age)
sportMean <- cfeat %>% group_by(Sport) %>% mutate(sportAvgHeight = mean(Height)) %>% mutate(sportAvgWeight = mean(Weight))
sportMean <- distinct(sportMean,Sport, .keep_all = TRUE)
cfeat2 <- cfeat %>% group_by(Sport) %>% sample_n(10, replace =  TRUE)#[sample(nrow(cfeat), 2000),]
plt <- cfeat %>% ggplot() + geom_point(aes(color = as.factor(labels), x = Height, y = Weight, text = paste("Sport: ", Sport, ", Year:", Year, ", Age:", Age)))
plt
#plt <- cfeat2 %>% ggplot() + geom_point(aes(color = as.factor(labels), x = Height, y = Weight, text = paste("Sport: ", Sport, ", Year:", Year, ", Age:", Age)))
#ggplotly(plt,, hover = c("plt$text"))
sportMean %>% ggplot() + geom_label_repel(aes(label = Sport,x = sportAvgHeight, y = sportAvgWeight), size = 4)

20 clusters is too many to keep track of, to find a optimum number for k we can use various algorithms such as... elbow method, silhouette method

For this we can use the silhouette method.

In [ ]:

nbc <- NbClust(data = scale(cfeat[,c(1:4)]), method="kmeans", index = "silhouette")
nbc$Best.nc
clust <- kmeans(scale(cfeat[,c(1:4)]), 2)
labels2 <- clust$cluster
cfeat <- cfeat %>% mutate(labels = labels2) %>% mutate(Year = features$Year) %>% mutate(Sport = features$Sport) %>% mutate(Age = features$Age)
plt <- cfeat %>% ggplot() + geom_point(aes(color = as.factor(labels2), x = Height, y = Weight, text = paste("Sport: ", Sport, ", Year:", Year, ", Age:", Age)))
plt

According to silhouette method, the optimal number of k is 2.

From BMI and sex perspectives... the multiplication by 10000 (100^2) is due to the height being presented in centimeters.

In [ ]:
cfeat <- cfeat %>% mutate(BMI = 10000*Weight/(Height*Height))
plt <- cfeat %>% ggplot() + geom_point(aes(color = as.factor(labels2), x = Age, y = BMI, text = paste("Sport: ", Sport, ", Year:", Year, ", Age:", Age)), alpha = 0.7)
plt
plt <- cfeat %>% ggplot() + geom_jitter(aes(color = as.factor(labels2), x = Sex, y = Age), alpha = 0.7)
plt

Looks like K-means preferred to prioritize Sex over the other attributes... Time to try DBscan for good measure.

Next to try : DBscan for good measure.

In [ ]:

clust <- hdbscan(scale(cfeat[,c(1:4)]), minPts = length(unique(cfeat$Sport)))
labels3 <- clust$cluster
plt <- cfeat %>% ggplot() + geom_point(aes(color = as.factor(labels3), x = Height, y = Weight, text = paste("Sport: ", Sport, ", Year:", Year, ", Age:", Age)))
plt

When HDB clustering method is implemented, there is a third cluster(represented in red!) 

TO DO : identify the red cluster

In [ ]:
plt <- cfeat %>% ggplot() + geom_point(aes(color = as.factor(labels3), x = Age, y = BMI, text = paste("Sport: ", Sport, ", Year:", Year, ", Age:", Age)), alpha = 0.7)
plt <- cfeat %>% ggplot() + geom_jitter(aes(color = as.factor(labels3), x = Sex, y = Age), alpha = 0.7)
plt

Red cluster  is representing some of the largest BMIs and ages. 
Although a few of them are exceptionally tall.
Let's analyse the sport they belong to.

In [ ]:
cfeat %>% mutate(labs = labels3) %>% filter(labs == 0)

The red class thus explains a few exceptional cases across all sports.

In [ ]:

dta <- olympics
print("Data before label encoding..\n")
head(dta)
label <- LabelEncoder$new()


dta$Name <- label$fit_transform(dta$Name)
dta$Sex <- label$fit_transform(dta$Sex)
dta$Team <- label$fit_transform(dta$Team)
dta$NOC <- label$fit_transform(dta$NOC)
dta$Games <- label$fit_transform(dta$Games)
dta$Season <- label$fit_transform(dta$Season)
dta$City <- label$fit_transform(dta$City)
dta$Sport <- label$fit_transform(dta$Sport)
dta$Event <- label$fit_transform(dta$Event)
dta$Medal <- label$fit_transform(dta$Medal)
print("Data after label encoding..\n")
head(dta)

In [ ]:
dtaold <- dta

In [ ]:
#Converting NA to 0 and (gold, silver, bronze) to 0 value for binomial classification
dta = dta %>% mutate(Medal = if_else(Medal == 0,0,1))
head(dta)

In [ ]:
print(length(unique(dta$Medal)))
print(length(unique(dta$Age)))
print(length(unique(dta$Name)))
print(length(unique(dta$Sex)))
print(length(unique(dta$Team)))
print(length(unique(dta$NOC)))
print(length(unique(dta$Games)))
print(length(unique(dta$Season)))
print(length(unique(dta$City)))
print(length(unique(dta$Sport)))
print(length(unique(dta$Event)))

In [ ]:
dta2 <- dta #for naive bayes algorithm
dta3 <- dta #for SVM

In [ ]:
set.seed(15)
tr_samples <- dta$Medal %>%
                 createDataPartition(p=0.8,list = FALSE)
train_data <- dta[tr_samples,]
test_data <- dta[-tr_samples,]

Multiple Regression Model

In [ ]:
model <- glm(Medal~.,data=train_data,family = "binomial")
summary(model)

Predicting Test Data

In [ ]:
probability_list <- model %>%
         predict(test_data,type="response")
head(probability_list)

In [ ]:
predicted_class <- ifelse(probability_list<0.5,0,1)
head(predicted_class)

Model Accuracy

In [ ]:
test_data = test_data %>% mutate(Medal = ifelse(Medal<0.5,0,1))


print(paste("Accuracy =",mean(predicted_class==test_data$Medal)*100,"%",sep =" "))

Multiple Regression Model - (Multinominal)

- to check if we can predict the medal type as well.

In [ ]:
set.seed(15)
tr_samples <- dtaold$Medal %>%
           createDataPartition(p=0.8,list=FALSE)
train_data <- dtaold[tr_samples,]
test_data <- dtaold[-tr_samples,]

mulnorm_model <- multinom(Medal~.,data = dtaold)

In [ ]:
summary(mulnorm_model)

In [ ]:
#predictions
predicted_class <- mulnorm_model%>%predict(test_data)
head(predicted_class)

Modal Accuracy

In [ ]:
#model accuracy
logisticAccuracy = mean(predicted_class==test_data$Medal)*100
print(paste("Accuracy =",logisticAccuracy,"%",sep =" "))

Naive Bayes Algorithm

In [ ]:
set.seed(15)
tr_samples <- dta2$Medal %>%
                 createDataPartition(p=0.8,list = FALSE)
train_data <- dta2[tr_samples,]
test_data <- dta2[-tr_samples,]

In [ ]:
NBmodel = naiveBayes(Medal~.,data = dta2)
NBmodel

Predicting

In [ ]:
ypred = NBmodel %>% predict(dta2)

Accuracy :

In [ ]:
mean(ypred == dta2$Medal)*100

In [ ]:
#confusionMatrix(ypred,dta2$Medal)

#can include 10 fold cross validation for increased accuracy


.
.
.

#modelCrossValidation = train(quality~.,data = df,method="nb",trControl = trainControl("cv",number = 10))

SVM - Support Vector Machine

In [ ]:
#SVM using non-linear kernel-RBF
set.seed(15)
#model <- train(Medal~.,data=train_data,method="svmRadial",trControl=trainControl("cv",number=10),preProcess=c("center","scale"),tuneLength=3)
#model$bestTune

In [ ]:
#predicted <- model%>%predict(test_data)

Accuracy

In [ ]:
#print(mean(predicted==test_data$Medal)*100)

3 Classification algorithms used with their respective accuracy :

1. Logistic Regression (binominal)      : 85.2066715010878 %
2.   Logistic Regression (multinominal) : 85.2284263959391 %
3. Naive Bayes                           : 84.2989949384345 %
4. Support Vector Machine (SVM)          : 73.4690111758227%

Conclusions for the research questions :

1. How do various strategies undertaken by various countries contribute to their respective winnings ?
...India has put into motion a 14-sport olympic strategy. Following which the tokyo olympics was conducted. Although it is fairly early to determine the effect of this strategy, the results have indeed proven to have an positive impact. 

2. To what extent does the host country have an advantage in the medals won at the Olympics?

3. How baised is the Olympics? Is there a significant difference in the sex ratio of participants and winners and How has it changed over the years?
...The sex ratio is very biased towards the male population. Although in the recent years the number of women participating has taken an upward toll, yet no match to the number of male participants. But it is to be noted that the participation to the participants ratio is higher in the case of females indicating that although there are less number of woman participating, each woman on an average is participating/winning in more events as compared to men in olympics.

4. Is there an age group for winning the Olympics, or not? Or has it changed over the years?
...Although there are significant outliers, majority of the winners and participants belong to the age group of 23 to 37. And this has been consistent throughout. 

5. Is there a consistency in events/ countries won?
...Over the years, United states successfully clutched on to the position of the country with the highest number of Gold medals for decades.Although, Soviet Union provided a tough competition. Lately, we see the emergence of Canada securing the highest number of Medals in Winter Olympics.

6. Is any country dominating the Olympics?
...Yes, United States has been ruling over the olympics for decades now. Although in recent years we see the emergence of new courtries in the top radar.

7. Is there a pattern or a certain set of characteristics that might be help cluster the participants in the olympcis?
...yes, the promising set of attributes that have been able to cluster the participants into clusters are sex, BMI and miscellanous exceptions.
(obtained used various clustering algorithms including K-Means, Hierarchical Based, DBScan and methods like Silhouette method to find the optimal number of clusters)

8. How did COVID affect the summer Olympics held in 2021?
...The amount of precausions taken before the commencement of Olympics encouraged participants from all over the globe to participate. Although the numbers seen aren't significant enough.

9. How has the Olympics changed (growth) over the years?
... Yes, the olympics has definitely improved over the years. The number of sports, events, and participants has been constantly increasing ever since the inception of the Olympics Games.


10. Has India’s participation and performance in the Olympics over the years improved?
...Although India set some exceptionally high benchmarks a few certain years, over the few years the performance of the country has been on the down low. Nevertheless, India showed promising results in the Tokyo Olympics.